# Diffuse model computation

This notebook shows how to compute the follwing images:
* integral surface brightness (cm^-2 s^-1 sr^-1)
* expected counts (counts / bin) image assuming a constant exposure

In [2]:
import numpy as np
from astropy.io import fits
from astropy.units import Quantity

## Compute diffuse integral surface brightness image

In [5]:
from gammapy.cube import SkyCube
filename = '$GAMMAPY_EXTRA/datasets/source_diffuse_separation/galactic_simulations/gll_iem_v06.fits'
cube = SkyCube.read(filename)
print(cube)

energy_band = Quantity([10, 500], 'GeV').to('MeV')
flux = cube.integral_flux_image(energy_band, energy_bins=10)
# Note: "flux" means surface brightness in units of 'cm^-2 s^-1 sr^-1' here.
flux.writeto('diffuse_integral_flux.fits', clobber=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jer/git/gammapy-extra/datasets/source_diffuse_separation/galactic_simulations/gll_iem_v06.fits'

In [ ]:
print(flux.data[1441/2, 2880/2]) # 8.805560757e-09

## Reproject to your image

In [ ]:
from kapteyn.maputils import FITSimage
flux_image = FITSimage('diffuse_integral_flux.fits', hdunr=1)
reference_image = FITSimage('10_500_counts.fits')
flux_image2 = flux_image.reproject_to(reference_image)
flux_image2.writetofits('diffuse_integral_flux_reprojected.fits', clobber=True)

## Diffuse expected counts image

In [ ]:
#exposure = Quantity(1.5768e11, 'cm^2 s')
#surface_brightness = Quantity(image.data.mean(), 'cm^-2 s^-1 sr^-1')
#print('Mean surface brightness in image: {0}'.format(surface_brightness))

header = fits.getheader('10_500_counts.fits')

exposure = fits.getdata('exposure_10_GeV.fits')
exposure = Quantity(exposure, 'cm^2 s')

flux = fits.getdata('diffuse_integral_flux_reprojected.fits')
flux = Quantity(flux, 'cm^-2 s^-1 sr^-1')

pixel_size = Quantity(0.1, 'deg') ** 2


####
####

expected_counts = (exposure * pixel_size * flux).to('').value

####
####



fits.writeto(filename='diffuse_counts.fits', data=expected_counts, header=header, clobber=True)

# Compute some numbers to check if this looks OK

In [ ]:
(0.1 * Quantity(5, 'year') * Quantity(1, 'm^2')).to('cm^2 s')

In [ ]:
# TODO: Counts should roughly match Fermi counts, which is 1 count / pixel in the Galactic plane.
